In [1]:
# Тренировачный вариант
import urllib.parse
import urllib.request

import requests
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup 

import csv
import sys
import time
import lxml

from urllib.request import Request, urlopen
from fake_useragent import UserAgent
from unicodedata import normalize


In [2]:
def parser_cars(url, hdr):
    # создаем запрос
    response = requests.get(url, headers = hdr)    
    # Спим 5 секунд
    #time.sleep(3)
    # Применяем bs
    soup = BeautifulSoup(response.content, 'html.parser')
    # Декодируем страницу
    soup.encoding = 'utf-8'
    # Пишем парсер страницы
    cars_df = []
    cars_df.append({'bodyType': soup.find_all('li', class_ = 'CardInfoRow CardInfoRow_bodytype')[0].text[5:],
                    'brand': soup.find_all('div', class_ = 'CardBreadcrumbs__item')[2].text,
                    'color': soup.find_all('li', class_ = 'CardInfoRow CardInfoRow_color')[0].text[4:]})
    
    # Тут добавим условия аотому что не везде комплектация есть               
    try: cars_df[0].update({'complectation_dict': soup.find_all('h2', class_ = 'CardComplectation__title')[0].text[13:]})
    except: cars_df[0].update({'complectation_dict':None})
    try: cars_df[0].update({'description': normalize('NFKD',soup.find_all('div', class_ = 'CardDescription__text')[0].text)})
    except: cars_df[0].update({'description':None})    
        
        
    # Продолжаем парсить страницу         
    cars_df[0].update({'engineDisplacement': normalize('NFKD',soup.find_all('li', class_ = 'CardInfoRow CardInfoRow_engine')[0].text[9:]),
                     'equipment_dict': None})
                       
                       
    try: cars_df[0].update({'fuelType': soup.find_all('a', class_ = 'Link Link_color_black')[3].text})
    except: cars_df[0].update({'fuelType': None})
                       
    cars_df[0].update({'image': None,
                    'mileage': normalize('NFKD',soup.find_all('li', class_ = 'CardInfoRow CardInfoRow_kmAge')[0].text[6:-3]),
                    'modelDate': None,
                    'model_name': soup.find_all('div', class_ = 'InfoPopup InfoPopup_theme_plain InfoPopup_withChildren BreadcrumbsPopup')[1].text,
                    'name': soup.find_all('div', class_ = 'InfoPopup InfoPopup_theme_plain InfoPopup_withChildren BreadcrumbsPopup')[4].text,
                    'numberOfDoors': None,
                    'parsing_unixtime': round(time.time()),
                    'priceCurrency': None,
                    'productionDate': soup.find_all('li', class_ = 'CardInfoRow CardInfoRow_year')[0].text[11:],
                    'sell_id': soup.find_all('div', class_ = 'CardHead__infoItem CardHead__id')[0].text[2:],
                    'super_gen': None,
                    'vehicleConfiguration': None,
                    'vehicleTransmission': soup.find_all('li', class_ = 'CardInfoRow CardInfoRow_transmission')[0].text[7:],
                    'vendor': None,
                    'Владельцы': soup.find_all('li', class_ = 'CardInfoRow CardInfoRow_ownersCount')[0].text[9:10]
                   })
                
    # Тут добавим условия аотому что не везде владение есть
    try: cars_df[0].update({'Владение': soup.find_all('li', class_ = 'CardInfoRow CardInfoRow_owningTime')[0].text[8:]})
    except: cars_df[0].update({'Владение':None})
                    
    # Продолжаем парсить страницу               
    cars_df[0].update({'ПТС': soup.find_all('li', class_ = 'CardInfoRow CardInfoRow_pts')[0].text[3:],
                    'Привод': soup.find_all('li', class_ = 'CardInfoRow CardInfoRow_drive')[0].text[6:],
                    'Руль': soup.find_all('li', class_ = 'CardInfoRow CardInfoRow_wheel')[0].text[4:],
                    'Состояние': soup.find_all('li', class_ = 'CardInfoRow CardInfoRow_state')[0].text[9:],
                    'Таможня': soup.find_all('li', class_ = 'CardInfoRow CardInfoRow_customs')[0].text[7:] })
                       
                       
    # Тут добавим условия аотому что не везде владение есть цена
    try: cars_df[0].update({'Price': normalize('NFKD', soup.find_all('div', class_ = 'PriceUsedOffer')[0].text[:-25])})
    except: cars_df[0].update({'Price':None})
                  
                           
        
    return cars_df



In [3]:
def parser_url(url, hdr):
    # создаем запрос
    response = requests.get(url, headers = hdr)    
    # Применяем bs
    soup = BeautifulSoup(response.content, 'html.parser')
    soup.encoding = 'utf-8'
    
    items = soup.find_all("div", class_ = "ListingItem")
    time.sleep(2)
    cars = []
    for i in items:
        
        cars.append(i.find('a', class_ = 'Link ListingItemTitle__link').get('href')
                        
                        )
    return cars

In [247]:
url = 'https://auto.ru/cars/mercedes/used/body-limousine/?sort=cr_date-desc&output_type=table?'
hdr = { 'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)' }

In [248]:
df = []
a = 0

for page in range (1,2):
    print(f'Парсинг страницы {page} из 99...')
    car_url = parser_url(f'{url}={page}',hdr)
    for i in car_url:
        a += 1
        print(f'Автомобиль {a}')
        df.extend(parser_cars(i,hdr))

Парсинг страницы 1 из 99...
Автомобиль 1
Автомобиль 2
Автомобиль 3
Автомобиль 4
Автомобиль 5
Автомобиль 6


In [249]:
url_train = pd.DataFrame(df)

In [251]:
url_train.sample(1)

,bodyType,brand,color,complectation_dict,description,engineDisplacement,equipment_dict,fuelType,image,mileage,...,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,Price
2,лимузин,Mercedes-Benz,чёрный,,Комплектация:Датчик давления в шинахДатчик про...,6.0 л / 530 л.с. / Бензин,None,Бензин,None,2 243,...,автоматическая,None,1,None,Оригинал,задний,Левый,Не требует ремонта,Растаможен,45 000 000


In [252]:
len(url_train)

6

In [253]:
url_train.to_csv('train.csv', mode='a')

In [254]:
train = pd.read_csv('train.csv',encoding='utf-8')